<a href="https://colab.research.google.com/github/Addechander/fcc-cats-and-dogs-image-classifier/blob/main/predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
numeric_columns = ['age', 'bmi', 'children', 'expenses']

ohe_sex = pd.get_dummies(dataset[['sex']])
ohe_dataset = pd.concat([dataset[numeric_columns], ohe_sex], axis = 1)

ohe_smoker = pd.get_dummies(dataset[['smoker']])
ohe_dataset = pd.concat([ohe_dataset, ohe_smoker], axis = 1)

ohe_region = pd.get_dummies(dataset[['region']])
ohe_dataset = pd.concat([ohe_dataset, ohe_region], axis = 1)

from sklearn.model_selection import train_test_split

independent_var = ohe_dataset[ohe_dataset.columns.drop('expenses')].values
dependent_var = ohe_dataset['expenses'].values

train_dataset, test_dataset, train_labels, test_labels = train_test_split(independent_var, dependent_var, train_size=0.8, test_size = 0.2)

In [ ]:
normalizer = layers.experimental.preprocessing.Normalization()
normalizer.adapt(np.array(train_dataset))

model = keras.Sequential([
    normalizer,
    layers.Dense(16),
    layers.Dense(4),
    layers.Dropout(.2),
    layers.Dense(1),
])

model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mae',
    metrics=['mae', 'mse'])

In [ ]:
history = model.fit(
    train_dataset,
    train_labels,
    epochs=100,
    validation_split=0.5,
    verbose=0,
)


In [ ]:
model.evaluate(test_dataset, test_labels)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
